In [24]:
from datetime import timedelta, datetime
from django.conf import settings
import os
import glob
from time import mktime
import logging
logger = logging.getLogger(__name__)

def timestamp(delta, fallback):
    """Calculates seconds since epoch of current time minus delta.
    If delta is out of range, calculates timestamp of fallback datetime instead.
    """
    try:
        dt = datetime.now() - delta
    except (OverflowError):
        # logger.exception('locals:%s' % locals())
        dt = fallback
    return int(mktime(dt.timetuple()))

def new_staging_files(staging_subdirectory, fileglob='*.*', min_age=timedelta.min, max_age=timedelta.max):
    """Check for new or updated files in staging area."""
    globpattern = os.path.join(settings.STAGING_ROOT, staging_subdirectory, fileglob,)
    all_files = glob.glob(globpattern)
    min_mtime = timestamp(min_age, fallback=datetime.max)
    max_mtime = timestamp(max_age, fallback=datetime.min)
    files = [file for file in all_files 
             if min_mtime > os.path.getmtime(file) > max_mtime]
    return sorted(files)

#staging_files = new_staging_files('IMAGES', '*.jpg', min_age=timedelta(days=30))
#len(staging_files)
# new_staging_files('BYLINE', '*.jpg')
#timedelta_to_timestamp(timedelta(days=-10000*365), datetime.min)

def new_staging_images(**kwargs):
    """Check for new or updated images in staging area"""
    arguments = dict(
        staging_subdirectory='IMAGES',
        fileglob='*.jpg',
        max_age=timedelta(days=7),
    )
    arguments.update(kwargs)
    return new_staging_files(**arguments)

def new_staging_byline_images(**kwargs):
    """Check for new or updated byline images in staging area"""
    arguments = dict(
        staging_subdirectory='BYLINE',
        fileglob='*.jpg',
        max_age=timedelta(days=1),
    )
    arguments.update(kwargs)
    return new_staging_files(**arguments)

def new_staging_pdf_files(**kwargs):
    """Check for new or updated pdf files in staging area"""
    arguments = dict(
        staging_subdirectory='PDF',
        fileglob='UNI1*VER*000.pdf',
        max_age=timedelta(days=100),
    )
    arguments.update(kwargs)
    return new_staging_files(**arguments)

print(
    new_staging_byline_images(),
    '\n\n',
    new_staging_pdf_files(),
    '\n\n',
    new_staging_images(),
)

['/srv/fotoarkiv_universitas/STAGING/BYLINE/Zomlot-Nora.jpg'] 

 ['/srv/fotoarkiv_universitas/STAGING/PDF/UNI11VER15101601000.pdf', '/srv/fotoarkiv_universitas/STAGING/PDF/UNI11VER15101602000.pdf', '/srv/fotoarkiv_universitas/STAGING/PDF/UNI11VER15101603000.pdf', '/srv/fotoarkiv_universitas/STAGING/PDF/UNI11VER15101604000.pdf', '/srv/fotoarkiv_universitas/STAGING/PDF/UNI11VER15101605000.pdf', '/srv/fotoarkiv_universitas/STAGING/PDF/UNI11VER15101606000.pdf', '/srv/fotoarkiv_universitas/STAGING/PDF/UNI11VER15101607000.pdf', '/srv/fotoarkiv_universitas/STAGING/PDF/UNI11VER15101608000.pdf', '/srv/fotoarkiv_universitas/STAGING/PDF/UNI11VER15101609000.pdf', '/srv/fotoarkiv_universitas/STAGING/PDF/UNI11VER15101610000.pdf', '/srv/fotoarkiv_universitas/STAGING/PDF/UNI11VER15101611000.pdf', '/srv/fotoarkiv_universitas/STAGING/PDF/UNI11VER15101612000.pdf', '/srv/fotoarkiv_universitas/STAGING/PDF/UNI11VER15101613000.pdf', '/srv/fotoarkiv_universitas/STAGING/PDF/UNI11VER15101614000.pdf', '/srv/foto

In [4]:
dict(a=1) | dict(b=2)

TypeError: unsupported operand type(s) for |: 'dict' and 'dict'

In [ ]:
dict.update

In [29]:
def sna(bacon):
    return bacon[::-1]

class Foo():
    sna = staticmethod(sna)

foo = Foo()
foo.sna('bacon')

'nocab'

In [26]:
{datetime.now():'now'}

{datetime.datetime(2015, 10, 19, 22, 25, 7, 570734): 'now'}

In [ ]:
datetime.min.strftime('%s')

In [ ]:
import os
from datetime import datetime

now = datetime.now()
IMAGE_STAGING = os.path.join(settings.STAGING_ROOT, 'STAGING', 'IMAGES')
os.chdir(IMAGE_STAGING)
print(os.path.abspath(os.curdir))

In [ ]:
for file_path in sorted(os.listdir(), key=lambda f: os.path.getmtime(f), reverse=True): 
#    exists = ImageFile.objects.filter(old_file_path=file_path)
#    if exists:
#        print(exists)
#        continue
    mtime = os.path.getmtime(file_path)
    modified = datetime.fromtimestamp(mtime)
    print('{0:<40.40}  {1}'.format(file_path, modified))
    